## 세계 CPU and GPU성능에 따른 대한민국 반도체 및 디스플레이 동향파악

1. CPU와 GPU를 년도별로 정리 및 주파수 and 트랜지스터 and processing size 비교
> 하나의 pd로
2. 대한민국 반도체 시장이 CPU와 GPU에 얼마나 연관성이 있냐., DRAM
3. 대한민국 디스플레이 시장이 CPU와 GPU에 얼마나 연관성이 있냐.
4. 최종 : 스마트폰 가격을 알 수 있냐?

> 전처리 :  
           
           1. NaN값 확인 필요없는 값 drop, 
           
           2. Release Date를 정제하기(년도 만 빼내기)
           
           3. 반도체 및 디스플레이 데이터와 합치기
           
           
위키피아 휴대폰 모델 검색

> CPU와 GPU가 무어의 법칙을 따르는가 - 무어의 법칙을 설명해야함

> Dennard scaling 법칙을 따르는가 - Dennard scaling를 설명해야함

> 시점 구간별 정리 제품 출시업체(CPU = intel, amd, GPU = NVIDIA, intel)

> CPU와 GPU의 년간 Freq와 TDP의 상관도가 있는가

- GPU 성능 향상은 더 작은 트랜지스터, 더 큰 다이 크기 및 더 높은 주파수의 공동 효과
- 주파수에 따른 CPU와 GPU차이
- 각 회사 별 GPU 주파수, 트랜지스터 차이
- 아무래도 년도가 같을 수록 서로 잘 맞을 것이다. cpu나 gpu를 알려주는
- 트랜지스터나 주파수를 치면 대충 어느정도의 성능이 나올 것인지 (회사입장)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait

In [2]:
def check_Date(df):

    return df[:4]

In [3]:
file = 'chip_dataset.csv'
pd_chip = pd.read_csv(file, encoding = 'UTF-8')
cp_chip = pd_chip
cp_chip.head()

,Unnamed: 0,Product,Type,Release Date,Process Size (nm),TDP (W),Die Size (mm^2),Transistors (million),Freq (MHz),Foundry,Vendor,FP16 GFLOPS,FP32 GFLOPS,FP64 GFLOPS
0,0,AMD Athlon 64 3500+,CPU,2007-02-20,65.0,45.0,77.0,122.0,2200.0,Unknown,AMD,NaN,NaN,NaN
1,1,AMD Athlon 200GE,CPU,2018-09-06,14.0,35.0,192.0,4800.0,3200.0,Unknown,AMD,NaN,NaN,NaN
2,2,Intel Core i5-1145G7,CPU,2020-09-02,10.0,28.0,NaN,NaN,2600.0,Intel,Intel,NaN,NaN,NaN
3,3,Intel Xeon E5-2603 v2,CPU,2013-09-01,22.0,80.0,160.0,1400.0,1800.0,Intel,Intel,NaN,NaN,NaN
4,4,AMD Phenom II X4 980 BE,CPU,2011-05-03,45.0,125.0,258.0,758.0,3700.0,Unknown,AMD,NaN,NaN,NaN


In [4]:
# 필요없는 부분 drop
cp_chip.drop(['Unnamed: 0','TDP (W)', 'Die Size (mm^2)', 'Product', 'FP16 GFLOPS', 'FP32 GFLOPS', 'FP64 GFLOPS'], axis = 1, inplace = True)

In [5]:
# 년도만
cp_chip['년도'] = cp_chip['Release Date'].apply(check_Date)

In [6]:
# 원래 Release Date 삭제
cp_chip.drop(['Release Date'], axis = 1, inplace = True)

In [7]:
# NaN값 대체, 소수점 다 버림
Tran_mean = round(cp_chip['Transistors (million)'].mean(), 0)
Freq_mean = round(cp_chip['Freq (MHz)'].mean(), 0)
Process_mean = round(cp_chip['Process Size (nm)'].mean(), 0)


cp_chip['Transistors (million)'] = cp_chip['Transistors (million)'].fillna(Tran_mean)
cp_chip['Freq (MHz)'] = cp_chip['Freq (MHz)'].fillna(Freq_mean)
cp_chip['Process Size (nm)'] = cp_chip['Process Size (nm)'].fillna(Process_mean)
cp_chip

,Type,Process Size (nm),Transistors (million),Freq (MHz),Foundry,Vendor,년도
0,CPU,65.0,122.0,2200.0,Unknown,AMD,2007
1,CPU,14.0,4800.0,3200.0,Unknown,AMD,2018
2,CPU,10.0,1930.0,2600.0,Intel,Intel,2020
3,CPU,22.0,1400.0,1800.0,Intel,Intel,2013
4,CPU,45.0,758.0,3700.0,Unknown,AMD,2011
...,...,...,...,...,...,...,...
4849,GPU,40.0,1950.0,450.0,TSMC,NVIDIA,2011
4850,GPU,90.0,1930.0,250.0,Intel,Intel,2005
4851,GPU,40.0,486.0,500.0,TSMC,NVIDIA,2010
4852,GPU,150.0,29.0,250.0,TSMC,NVIDIA,2003


In [8]:
# 각연도 별 groupby mean값 (연도와 type으로) 없는 index는 drop
g1 = cp_chip.groupby(['년도', 'Type'])
g2 = g1.mean()
g3 = g2.drop([('2000', 'CPU'),
            ('2000', 'GPU'),
            ('2001', 'CPU'),
            ('2001', 'GPU'),
            ('2002', 'CPU'),
            ('2002', 'GPU'),
            ('2003', 'CPU'),
            ('2003', 'GPU')], axis = 0)
g4 = g3.unstack()
g4.columns = ['Process Size(nm)_CPU', 'Process Size(nm)_GPU', 'Transistors (million)_CPU',
              'Transistors (million)_GPU', 'Freq (MHz)_CPU', 'Freq (MHz)_GPU']
g4
g5 = g4.drop('NaT', axis = 0)
g5

/tmp/ipykernel_3044/2841394540.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  g2 = g1.mean()


,Process Size(nm)_CPU,Process Size(nm)_GPU,Transistors (million)_CPU,Transistors (million)_GPU,Freq (MHz)_CPU,Freq (MHz)_GPU
년도,,,,,,
2004,109.705882,121.384615,104.213235,332.938462,2267.705882,367.376923
2005,92.954545,112.315789,230.584416,294.684211,2409.831169,405.273684
2006,75.925926,93.178295,478.266667,348.031008,2370.696296,460.472868
2007,70.869565,80.031646,398.014493,388.797468,2180.652174,524.360759
2008,58.571429,63.439306,606.089286,566.826590,2287.241071,564.190751
2009,45.913043,52.783019,533.860870,613.830189,2519.904348,584.584906
2010,40.576389,42.870748,539.958333,972.979592,2477.243056,613.741497
2011,33.465409,39.280702,976.616352,1228.198830,2355.496855,641.356725
2012,27.455696,33.653179,1602.253165,1685.982659,2604.468354,696.046243


In [9]:
data = '산업통상자원부_반도체디스플레이 산업 동향_20211231.csv'
pd_con = pd.read_csv(data, encoding = 'cp949')
pd_con = pd_con.set_index('연도')
pd_con

,반도체_생산(조원),반도체_시장점유율(퍼센트),반도체_수출(억불),반도체_수출증가율(퍼센트),반도체_수입(억불),반도체_수입증가율(퍼센트),DRAM_가격(달러),디스플레이_생산(조원),디스플레이_시장점유율(퍼센트),디스플레이_수출(억불),디스플레이_수출증가율(퍼센트),디스플레이_수입(억불),디스플레이_수입증가율(퍼센트),LCD_평균가격(달러)
연도,,,,,,,,,,,,,,
2004,34.6,9.6,265.2,35.7,236.2,10.7,1.72,33.0,38.7,14,75.0,10,42.9,28.9
2005,32.4,10.5,299.8,13.1,251.3,6.4,2.16,36.0,40.7,49,250.0,10,0.0,28.0
2006,32.4,11.3,373.6,24.6,280.4,11.6,1.59,40.6,40.6,125,155.1,12,20.0,25.8
2007,37.5,11.4,390.4,4.5,308.2,9.9,2.17,43.6,40.4,171,36.8,30,150.0,23.8
2008,36.4,9.8,327.9,-16.0,320.2,3.9,0.93,51.9,39.5,188,9.9,39,30.0,30.1
2009,40.8,11.7,310.4,-5.3,266.2,-16.9,0.66,72.6,45.3,256,36.2,42,7.7,28.6
2010,51.2,14.3,507.1,63.4,311.4,17.0,1.20,90.0,45.7,332,29.7,52,23.8,31.9
2011,61.5,13.8,501.4,-1.1,324.8,4.3,0.60,86.7,47.6,333,0.3,51,-1.9,30.1
2012,64.5,14.3,504.3,0.6,322.4,-0.8,1.60,82.3,50.7,368,10.5,45,-11.8,34.7


In [10]:
pd_con.index = g5.index
pd.concat((pd_con, g5), axis = 1)

,반도체_생산(조원),반도체_시장점유율(퍼센트),반도체_수출(억불),반도체_수출증가율(퍼센트),반도체_수입(억불),반도체_수입증가율(퍼센트),DRAM_가격(달러),디스플레이_생산(조원),디스플레이_시장점유율(퍼센트),디스플레이_수출(억불),디스플레이_수출증가율(퍼센트),디스플레이_수입(억불),디스플레이_수입증가율(퍼센트),LCD_평균가격(달러),Process Size(nm)_CPU,Process Size(nm)_GPU,Transistors (million)_CPU,Transistors (million)_GPU,Freq (MHz)_CPU,Freq (MHz)_GPU
년도,,,,,,,,,,,,,,,,,,,,
2004,34.6,9.6,265.2,35.7,236.2,10.7,1.72,33.0,38.7,14,75.0,10,42.9,28.9,109.705882,121.384615,104.213235,332.938462,2267.705882,367.376923
2005,32.4,10.5,299.8,13.1,251.3,6.4,2.16,36.0,40.7,49,250.0,10,0.0,28.0,92.954545,112.315789,230.584416,294.684211,2409.831169,405.273684
2006,32.4,11.3,373.6,24.6,280.4,11.6,1.59,40.6,40.6,125,155.1,12,20.0,25.8,75.925926,93.178295,478.266667,348.031008,2370.696296,460.472868
2007,37.5,11.4,390.4,4.5,308.2,9.9,2.17,43.6,40.4,171,36.8,30,150.0,23.8,70.869565,80.031646,398.014493,388.797468,2180.652174,524.360759
2008,36.4,9.8,327.9,-16.0,320.2,3.9,0.93,51.9,39.5,188,9.9,39,30.0,30.1,58.571429,63.439306,606.089286,566.826590,2287.241071,564.190751
2009,40.8,11.7,310.4,-5.3,266.2,-16.9,0.66,72.6,45.3,256,36.2,42,7.7,28.6,45.913043,52.783019,533.860870,613.830189,2519.904348,584.584906
2010,51.2,14.3,507.1,63.4,311.4,17.0,1.20,90.0,45.7,332,29.7,52,23.8,31.9,40.576389,42.870748,539.958333,972.979592,2477.243056,613.741497
2011,61.5,13.8,501.4,-1.1,324.8,4.3,0.60,86.7,47.6,333,0.3,51,-1.9,30.1,33.465409,39.280702,976.616352,1228.198830,2355.496855,641.356725
2012,64.5,14.3,504.3,0.6,322.4,-0.8,1.60,82.3,50.7,368,10.5,45,-11.8,34.7,27.455696,33.653179,1602.253165,1685.982659,2604.468354,696.046243


In [11]:
dat = 'Mobile phone price.csv'
pd_phone = pd.read_csv(dat, encoding = 'cp949')
pd_phone

,癤풟rand,Model,Storage,RAM,Screen Size (inches),Camera (MP),Battery Capacity (mAh),Price ($)
0,Apple,iPhone 13 Pro,128 GB,6 GB,6.1,12 + 12 + 12,3095,999
1,Samsung,Galaxy S21 Ultra,256 GB,12 GB,6.8,108 + 10 + 10 + 12,5000,1199
2,OnePlus,9 Pro,128 GB,8 GB,6.7,48 + 50 + 8 + 2,4500,899
3,Xiaomi,Redmi Note 10 Pro,128 GB,6 GB,6.67,64 + 8 + 5 + 2,5020,279
4,Google,Pixel 6,128 GB,8 GB,6.4,50 + 12.2,4614,799
...,...,...,...,...,...,...,...,...
402,Samsung,Galaxy Note20 5G,128,8,6.7,12+64+12,4300,1049
403,Xiaomi,Mi 10 Lite 5G,128,6,6.57,48+8+2+2,4160,349
404,Apple,iPhone 12 Pro Max,128,6,6.7,12+12+12,3687,1099
405,Oppo,Reno3,128,8,6.4,48+13+8+2,4025,429


In [27]:
# Model로 스마트폰 출시년도를 가져오기 위해 selenium

driver = webdriver.Chrome()
driver.get('https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC')

# 돋보기 모양 클릭
search_button = driver.find_elements(By.XPATH, '//a[contains(@class, "toggle")]')
# 검색창 가져오기
search_box = driver.find_elements(By.XPATH, '//input[@class="vector-search-box-input"]')
print(search_box)


data = driver.find_elements(By.XPATH, '//td[@class = "searchResultImage-text"]//div[@class="searchresult"]')


search_button[0].click()
search_box[0].send_keys('Apple iPhone 13 Pro' + Keys.ENTER)
print(data)

for i in data:
    print(i.text)
#first_list[0].click()
#for i in pd_phone['Model']:
#    search_box[0].send_keys('i')

[<selenium.webdriver.remote.webelement.WebElement (session="9f38490fbad0edc78ca62939370029e5", element="759b9767-e2a5-4806-bcbe-3bb4a4f742bc")>]
[]


In [ ]:
<button class="cdx-button cdx-button--action-default cdx-button--weight-normal cdx-button--framed cdx-search-input__end-button">검색</button>

In [44]:
<input class="cdx-text-input__input" accesskey="f" autocapitalize="sentences" title="위키백과 검색 [alt-shift-f]" placeholder="위키백과 검색" aria-label="위키백과 검색" name="search" role="combobox" autocomplete="off" aria-autocomplete="list" aria-owns="cdx-typeahead-search-menu-0" aria-expanded="false" type="search">


[<selenium.webdriver.remote.webelement.WebElement (session="8994d2ba8f03f921d4a77790ee3ac810", element="1e3cc95b-2c51-4d63-bff4-b443577d65ca")>]


In [ ]:
<a href="/wiki/%EC%95%84%EC%9D%B4%ED%8F%B0_13_%ED%94%84%EB%A1%9C" title="아이폰 13 프로" data-serp-pos="0">아이폰 <span class="searchmatch">13</span> 프로</a>

In [ ]:
<input class="vector-search-box-input" type="search" name="search" placeholder="위키백과 검색" aria-label="위키백과 검색" autocapitalize="sentences" title="위키백과 검색 [alt-shift-f]" accesskey="f" id="searchInput" autocomplete="off">